In [ ]:
import os
import numpy as np
from scipy.io import loadmat

folder_path = r"..\los_data"  # 你的文件夹路径
file_list = [f for f in os.listdir(folder_path) if f.endswith('.mat')]  # 找到所有mat文件
file_list = sorted(file_list)  # 按文件名排序，比如 dev0.mat, dev1.mat, dev2.mat

data = []  # 用来存储每个mat文件中的 'data_Ineed'（转置后的）

for i, file_name in enumerate(file_list):
    full_path = os.path.join(folder_path, file_name)
    mat_contents = loadmat(full_path)
    if 'data_Ineed' in mat_contents:
        data_ineed = mat_contents['data_Ineed'].T  # 加上 .T 转置！
        
        # 数据格式转化，按 320 信号一块进行处理
        num_signals, num_samples = data_ineed.shape
        
        # 确保每 320 个信号作为一个大块进行处理
        block_size = 320
        num_blocks = num_signals // block_size  # 计算可以完整分成多少块

        # 舍去多余的信号
        data_ineed = data_ineed[:num_blocks * block_size, :]
        
        # 将信号数据重构：每 320 个信号为一个大块
        reshaped_data = data_ineed.reshape((block_size, num_blocks * num_samples)).T  # 转置回原来的格式

        data.append(reshaped_data)
    else:
        print(f"Warning: {file_name} 里没有 'data_Ineed' 变量！")

# 打印检查
for i, d in enumerate(data):
    print(f"第{i}个文件（{file_list[i]}）的数据 shape (处理后): {d.shape}")


In [ ]:
import numpy as np

# 假设data是从之前的加载代码中获取的
# data = <从前一部分代码中加载的数据>

SNR_dB = -20  # 设定信噪比（dB）

noisy_data = []  # 用来存储加噪声后的数据

print(f"=== 开始添加噪声 (SNR={SNR_dB}dB) ===")

for i, d in enumerate(data):
    # 检查信号类型（实数/复数）
    is_complex = np.iscomplexobj(d)
    
    if is_complex:
        print(f"文件{i}: 检测到复数信号，shape={d.shape}")
        
        # 计算复数信号功率
        signal_power = np.mean(np.abs(d) ** 2)
        
        # 计算噪声功率
        noise_power = signal_power / (10 ** (SNR_dB / 10))
        
        # 复数噪声：实部和虚部各占一半功率
        noise_std = np.sqrt(noise_power / 2)
        
        # 生成独立的实部和虚部噪声
        real_noise = np.random.normal(0, noise_std, d.shape)
        imag_noise = np.random.normal(0, noise_std, d.shape) * 1j
        
        # 合成复数噪声并添加到信号
        complex_noise = real_noise + imag_noise
        noisy_signal = d + complex_noise
        
    else:
        print(f"文件{i}: 实数信号，shape={d.shape}")
        
        # 计算信号的标准差
        signal_std = np.std(d)
        
        # 计算噪声的标准差，根据信噪比（SNR）
        noise_std = signal_std / (10 ** (SNR_dB / 20))  # 根据 SNR(dB) 计算噪声标准差
        
        # 生成高斯噪声并加到原信号上
        noise = np.random.normal(0, noise_std, d.shape)  # 均值为0，标准差为噪声标准差
        noisy_signal = d + noise  # 添加噪声
    
    noisy_data.append(noisy_signal)  # 保存加噪声后的信号

print("\n✅ 所有文件噪声添加完成！")

# 详细的验证和对比
print("\n" + "="*50)
print("噪声添加效果验证")
print("="*50)

for i, (orig, noisy) in enumerate(zip(data, noisy_data)):
    actual_noise = noisy - orig
    
    if np.iscomplexobj(orig):
        # 复数信号验证
        signal_power = np.mean(np.abs(orig) ** 2)
        noise_power = np.mean(np.abs(actual_noise) ** 2)
        actual_snr_db = 10 * np.log10(signal_power / noise_power) if noise_power > 0 else float('inf')
        
        print(f"\n文件{i} - 复数信号验证:")
        print(f"  📊 目标SNR: {SNR_dB}dB, 实际SNR: {actual_snr_db:.2f}dB")
        print(f"  ⚡ 信号功率: {signal_power:.6f}")
        print(f"  🔊 噪声功率: {noise_power:.6f}")
        print(f"  📏 实部噪声标准差: {np.std(actual_noise.real):.6f}")
        print(f"  📏 虚部噪声标准差: {np.std(actual_noise.imag):.6f}")
        
    else:
        # 实数信号验证
        signal_power = np.var(orig)
        noise_power = np.var(actual_noise)
        actual_snr_db = 10 * np.log10(signal_power / noise_power) if noise_power > 0 else float('inf')
        
        print(f"\n文件{i} - 实数信号验证:")
        print(f"  📊 目标SNR: {SNR_dB}dB, 实际SNR: {actual_snr_db:.2f}dB")
        print(f"  ⚡ 信号功率: {signal_power:.6f}")
        print(f"  🔊 噪声功率: {noise_power:.6f}")
        print(f"  📏 噪声标准差: {np.std(actual_noise):.6f}")

# 打印检查加噪声后的数据样本
print("\n" + "="*50)
print("数据样本详细对比")
print("="*50)

for i, (orig, noisy) in enumerate(zip(data, noisy_data)):
    print(f"\n📁 文件{i} - 前5个采样点对比:")
    
    if np.iscomplexobj(orig):
        # 复数信号样本对比
        print("  索引 | 原始信号 (实部+虚部j) | 加噪信号 (实部+虚部j) | 噪声 (实部+虚部j)")
        print("  " + "-" * 80)
        for j in range(min(5, orig.shape[0])):
            orig_val = orig[j, 0] if orig.ndim > 1 else orig[j]
            noisy_val = noisy[j, 0] if noisy.ndim > 1 else noisy[j]
            noise_val = actual_noise[j, 0] if actual_noise.ndim > 1 else actual_noise[j]
            
            print(f"  {j:2d}   | {orig_val.real:7.4f}{orig_val.imag:+.4f}j | "
                  f"{noisy_val.real:7.4f}{noisy_val.imag:+.4f}j | "
                  f"{noise_val.real:7.4f}{noise_val.imag:+.4f}j")
    else:
        # 实数信号样本对比
        print("  索引 | 原始信号 | 加噪信号 | 噪声")
        print("  " + "-" * 50)
        for j in range(min(5, orig.shape[0])):
            orig_val = orig[j, 0] if orig.ndim > 1 else orig[j]
            noisy_val = noisy[j, 0] if noisy.ndim > 1 else noisy[j]
            noise_val = actual_noise[j, 0] if actual_noise.ndim > 1 else actual_noise[j]
            
            print(f"  {j:2d}   | {orig_val:8.4f} | {noisy_val:8.4f} | {noise_val:8.4f}")

# 统计信息总结
print("\n" + "="*50)
print("添加噪声统计总结")
print("="*50)

total_files = len(data)
complex_files = sum(1 for d in data if np.iscomplexobj(d))
real_files = total_files - complex_files

print(f"📂 总文件数: {total_files}")
print(f"🔷 复数信号文件: {complex_files}")
print(f"🔶 实数信号文件: {real_files}")

# 检查SNR控制精度
snr_errors = []
for i, (orig, noisy) in enumerate(zip(data, noisy_data)):
    actual_noise = noisy - orig
    if np.iscomplexobj(orig):
        signal_power = np.mean(np.abs(orig) ** 2)
        noise_power = np.mean(np.abs(actual_noise) ** 2)
    else:
        signal_power = np.var(orig)
        noise_power = np.var(actual_noise)
    
    actual_snr_db = 10 * np.log10(signal_power / noise_power) if noise_power > 0 else float('inf')
    snr_error = abs(actual_snr_db - SNR_dB)
    snr_errors.append(snr_error)

avg_snr_error = np.mean(snr_errors)
max_snr_error = np.max(snr_errors)

print(f"🎯 SNR控制精度:")
print(f"  平均误差: {avg_snr_error:.4f} dB")
print(f"  最大误差: {max_snr_error:.4f} dB")
print(f"  {'✅ 优秀' if avg_snr_error < 0.1 else '⚠️ 良好' if avg_snr_error < 0.5 else '❌ 需要检查'}")

print(f"\n🎉 噪声添加完成！所有文件已处理并验证。")

In [ ]:
import numpy as np

def compute_doppler_shift(v, fc):
    """
    根据移动速度 v (m/s) 和载波频率 fc (Hz) 计算多普勒频移
    """
    c = 3e8  # 光速 m/s
    v = v/3.6 # 换算成m/s
    return (v / c) * fc

def add_doppler_shift(signal, fd, fs):
    """
    给IQ信号加多普勒频移
    signal: shape (num_channels, num_samples)，复数IQ信号
    fd: 多普勒频移 (Hz)
    fs: 采样率 (Hz)
    """
    num_channels, num_samples = signal.shape
    t = np.arange(num_samples) / fs  # 时间轴
    doppler_phase = np.exp(1j * 2 * np.pi * fd * t)  # 复指数
    return signal * doppler_phase

# 例子
fs = 20e6  # 采样率，比如1 MHz
fc = 2.4e9  # 载波频率，比如2.4 GHz Wi-Fi
v = 120  # 移动速度，(km/h)

fd = compute_doppler_shift(v, fc)  # 先计算多普勒频移
print(f"计算得到的多普勒频移 fd = {fd:.2f} Hz")

# 假设 data 是你之前读出来的list，每个是 shape (信号数量, 采样点数)
data_with_doppler = []
for sig in noisy_data:
    shifted_sig = add_doppler_shift(sig, fd, fs)
    data_with_doppler.append(shifted_sig)

print("添加多普勒频移后的信号：")
print(data_with_doppler)


In [ ]:
from torch.utils.data import DataLoader, TensorDataset, random_split, Subset
import numpy as np
import torch

# === 拼接数据 ===
all_data = []
all_labels = []

for device_idx, device_signals in enumerate(data_with_doppler):
    device_labels = np.full((device_signals.shape[0],), device_idx)
    all_data.append(torch.tensor(device_signals, dtype=torch.float32))
    all_labels.append(torch.tensor(device_labels, dtype=torch.long))

all_data = torch.cat(all_data, dim=0)
all_labels = torch.cat(all_labels, dim=0)

print(f"最终数据 shape: {all_data.shape}")
print(f"最终标签 shape: {all_labels.shape}")

dataset = TensorDataset(all_data, all_labels)

# === 固定划分训练集 + 测试集 ===
train_size = int(0.8 * len(dataset))  # 80% 训练
test_size = len(dataset) - train_size # 20% 测试
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

print(f"训练集大小: {len(train_dataset)}")
print(f"测试集大小: {len(test_dataset)}")


In [ ]:
# ==== 数据增强：对训练集添加高斯噪声 ====

def add_noise_to_dataset(dataset, noise_std=0.01):
    noisy_data = []
    noisy_labels = []
    for x, y in dataset:
        noise = torch.randn_like(x) * noise_std
        noisy_x = x + noise
        noisy_data.append(noisy_x)
        noisy_labels.append(y)
    noisy_data = torch.stack(noisy_data)
    noisy_labels = torch.tensor(noisy_labels)
    return TensorDataset(noisy_data, noisy_labels)

train_dataset = add_noise_to_dataset(train_dataset, noise_std=0.01)  # 可调节 std

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sns
from torch.utils.data import DataLoader, Subset
from datetime import datetime
from tqdm import tqdm
from sklearn.model_selection import KFold

# 检查 GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 假设 SNR_dB 和 fd 已经在之前定义
SNR_dB = globals().get('SNR_dB', 'no')
fd = globals().get('fd', 'no')

# 初始化模型参数
input_dim = 320
num_heads = 4
num_layers = 2
num_classes = 20
dropout = 0.3

# 训练参数
batch_size = 256
num_epochs = 200
learning_rate = 1e-4
weight_decay = 1e-4
patience = 5

timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
script_name = "cross"

folder_name = f"{timestamp}_{script_name}_SNR{SNR_dB}dB_fd{fd}_classes_{num_classes}_Transformer"
save_folder = os.path.join(os.getcwd(), "training_results", folder_name)
os.makedirs(save_folder, exist_ok=True)

results_file = os.path.join(save_folder, "results.txt")
with open(results_file, "w") as f:
    f.write(f"=== Experiment Summary ===\n")
    f.write(f"Feature Folder: trajectory_plots\n")
    f.write(f"Timestamp: {timestamp}\n")
    f.write(f"Total Classes: {num_classes}\n")
    f.write(f"SNR: {SNR_dB} dB\n")
    f.write(f"fd (Doppler shift): {fd} Hz\n")

# 定义SignalTransformer模型
class SignalTransformer(nn.Module):
    def __init__(self, input_dim, num_heads, num_layers, num_classes, dropout=0.1):
        super(SignalTransformer, self).__init__()
        self.embedding = nn.Linear(input_dim, input_dim)
        self.transformer = nn.Transformer(
            d_model=input_dim,
            nhead=num_heads,
            num_encoder_layers=num_layers,
            dropout=dropout,
            batch_first=True
        )
        self.fc = nn.Linear(input_dim, num_classes)
    
    def forward(self, x):
        x = self.embedding(x)
        x = self.transformer(x, x)
        if len(x.shape) == 3:
            x = x[:, -1, :]
        x = self.fc(x)
        return x

# K折交叉验证
n_splits = 5
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)
fold_results = []
test_results = []

test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

for fold, (train_idx, val_idx) in enumerate(kfold.split(train_dataset)):
    print(f"\n====== Fold {fold+1}/{n_splits} ======")
    train_subset = Subset(train_dataset, train_idx)
    val_subset = Subset(train_dataset, val_idx)

    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True, drop_last=True)
    val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False, drop_last=True)

    model = SignalTransformer(input_dim=input_dim, num_heads=num_heads,
                              num_layers=num_layers, num_classes=num_classes,
                              dropout=dropout).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []

    best_val_acc = 0
    patience_counter = 0
    best_model_wts = None
    best_val_loss = float('inf')
    patience_counter = 0

    for epoch in range(num_epochs):
        model.train()
        running_train_loss = 0.0
        correct_train, total_train = 0, 0
        
        with tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch") as tepoch:
            for batch_idx, (inputs, labels) in enumerate(tepoch):
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                
                running_train_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total_train += labels.size(0)
                correct_train += (predicted == labels).sum().item()
                train_accuracy = 100 * correct_train / total_train
                tepoch.set_postfix(train_loss=running_train_loss / (batch_idx + 1), train_accuracy=train_accuracy)

        epoch_train_loss = running_train_loss / len(train_loader)
        epoch_train_acc = 100 * correct_train / total_train
        train_losses.append(epoch_train_loss)
        train_accuracies.append(epoch_train_acc)

        # 验证
        model.eval()
        running_val_loss = 0.0
        correct_val, total_val = 0, 0
        all_preds, all_labels = [], []

        with torch.no_grad():
            with tqdm(val_loader, desc="Validation", unit="batch") as vepoch:
                for val_inputs, val_labels in vepoch:
                    val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)
                    val_outputs = model(val_inputs)
                    val_loss = criterion(val_outputs, val_labels)
                    running_val_loss += val_loss.item()
                    _, val_predicted = torch.max(val_outputs, 1)
                    total_val += val_labels.size(0)
                    correct_val += (val_predicted == val_labels).sum().item()
                    val_accuracy = 100 * correct_val / total_val
                    vepoch.set_postfix(val_loss=running_val_loss / (batch_idx + 1), val_accuracy=val_accuracy)


                    # 保存混淆矩阵数据
                    all_preds.extend(val_predicted.cpu().numpy())
                    all_labels.extend(val_labels.cpu().numpy())

        epoch_val_loss = running_val_loss / len(val_loader)
        epoch_val_acc = 100 * correct_val / total_val
        val_losses.append(epoch_val_loss)
        val_accuracies.append(epoch_val_acc)

        print(f"Epoch {epoch+1}/{num_epochs}: Train Loss: {epoch_train_loss:.4f}, Train Acc: {epoch_train_acc:.2f}%, Val Loss: {epoch_val_loss:.4f}, Val Acc: {epoch_val_acc:.2f}%")
        with open(results_file, "a") as f:
            f.write(f"Epoch {epoch+1} | Train Loss: {epoch_train_loss:.4f} | Train Acc: {epoch_train_acc:.2f}% | Val Loss: {epoch_val_loss:.4f} | Val Acc: {epoch_val_acc:.2f}%\n")

        # Early stopping
        if epoch_val_acc > best_val_acc:
            best_val_acc = epoch_val_acc
            best_val_loss = epoch_val_loss
            best_model_wts = model.state_dict()  # 保存最佳参数
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break

        scheduler.step()


    # ========= 恢复最优模型并测试 =========
    model.load_state_dict(best_model_wts)
    model.eval()

    # --- 在最优模型上重新跑验证集，得到混淆矩阵 ---
    all_preds, all_labels = [], []
    correct_val, total_val = 0, 0
    with torch.no_grad():
        for val_inputs, val_labels in val_loader:
            val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)
            val_outputs = model(val_inputs)
            _, val_predicted = torch.max(val_outputs, 1)
            total_val += val_labels.size(0)
            correct_val += (val_predicted == val_labels).sum().item()
            all_preds.extend(val_predicted.cpu().numpy())
            all_labels.extend(val_labels.cpu().numpy())

    best_val_acc = 100 * correct_val / total_val

    # --- 在最优模型上重新跑测试集 ---
    correct_test, total_test = 0, 0
    test_preds, test_labels_all = [], []
    with torch.no_grad():
        for test_inputs, test_labels in test_loader:
            test_inputs, test_labels = test_inputs.to(device), test_labels.to(device)
            outputs = model(test_inputs)
            _, predicted = torch.max(outputs, 1)
            total_test += test_labels.size(0)
            correct_test += (predicted == test_labels).sum().item()
            test_preds.extend(predicted.cpu().numpy())
            test_labels_all.extend(test_labels.cpu().numpy())

    test_acc = 100 * correct_test / total_test

    print(f"Fold {fold+1}: Best Val Acc={best_val_acc:.2f}%, Test Acc={test_acc:.2f}%")

    fold_results.append(best_val_acc)
    test_results.append(test_acc)

    # --- 绘制混淆矩阵（基于最优模型的验证集）---
    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(10,8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Predicted'); plt.ylabel('True'); plt.title(f'Fold {fold+1} Validation Confusion Matrix')
    plt.tight_layout()
    plt.savefig(os.path.join(save_folder, f"fold_{fold+1}_val_confusion_matrix.png"))
    plt.close()
    np.savetxt(os.path.join(save_folder, f"fold_{fold+1}_val_confusion_matrix.txt"), cm, fmt='%d')

    # --- 测试集混淆矩阵 ---
    cm_test = confusion_matrix(test_labels_all, test_preds)
    plt.figure(figsize=(10,8))
    sns.heatmap(cm_test, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Predicted'); plt.ylabel('True'); plt.title(f'Fold {fold+1} Test Confusion Matrix')
    plt.tight_layout()
    plt.savefig(os.path.join(save_folder, f"fold_{fold+1}_test_confusion_matrix.png"))
    plt.close()
    np.savetxt(os.path.join(save_folder, f"fold_{fold+1}_test_confusion_matrix.txt"), cm_test, fmt='%d')

    # 绘图
    plt.figure()
    plt.plot(range(1, len(train_losses)+1), train_losses, label='Train Loss')
    plt.plot(range(1, len(val_losses)+1), val_losses, label='Val Loss')
    plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.legend()
    plt.title(f'Fold {fold+1} Loss Curve')
    plt.savefig(os.path.join(save_folder, f"fold_{fold+1}_loss_curve.png"))
    plt.close()

    plt.figure()
    plt.plot(range(1, len(train_accuracies)+1), train_accuracies, label='Train Acc')
    plt.plot(range(1, len(val_accuracies)+1), val_accuracies, label='Val Acc')
    plt.xlabel('Epoch'); plt.ylabel('Accuracy'); plt.legend()
    plt.title(f'Fold {fold+1} Accuracy Curve')
    plt.savefig(os.path.join(save_folder, f"fold_{fold+1}_accuracy_curve.png"))
    plt.close()

# 汇总结果
avg_acc = np.mean(fold_results)
avg_test_acc = np.mean(test_results)
with open(results_file, "a") as f:
    for i in range(n_splits):
        f.write(f"Fold {i+1}: Val Acc={fold_results[i]:.2f}%, Test Acc={test_results[i]:.2f}%\n")
    f.write(f"\nAverage Val Accuracy: {avg_acc:.2f}%\n")
    f.write(f"Average Test Accuracy: {avg_test_acc:.2f}%\n")

print(f"\nAverage Val Acc={avg_acc:.2f}%, Average Test Acc={avg_test_acc:.2f}%")
